# Задание 1 (5 балла)

Имплементируйте алгоритм Леска (описание есть в семинаре) и оцените качество его работы на датасете `data/corpus_wsd_50k.txt`

В качестве метрики близости вы должны попробовать два подхода:

1) Jaccard score на множествах слов (определений и контекста)
2) Cosine distance на эмбедингах sentence_transformers

В качестве метрики используйте accuracy (% правильных ответов). Предсказывайте только многозначные слова в датасете

Контекст вы можете определить самостоятельно (окно вокруг целевого слова или все предложение). Также можете поэкспериментировать с предобработкой для обоих методов.

## 1. Загрузка и подготовка данных

Сначала скачаем корпус и преобразуем его в формат, с которым будет удобно работать. Так как нам нужны разные способы представления текстов для разных методов, сделаем список предложений, каждое из которых будет храниться в словаре со всеми необходимыми элементами. Добавим также словесные определения из Wordnet для многозначных слов

In [1]:
!wget https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/data/corpus_wsd_50k.txt.zip
!unzip -o corpus_wsd_50k.txt.zip

--2025-10-02 13:09:02--  https://github.com/mannefedov/compling_nlp_hse_course/raw/refs/heads/master/data/corpus_wsd_50k.txt.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/data/corpus_wsd_50k.txt.zip [following]
--2025-10-02 13:09:03--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/refs/heads/master/data/corpus_wsd_50k.txt.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4723095 (4.5M) [application/zip]
Saving to: ‘corpus_wsd_50k.txt.zip’

corpus_wsd_50k.txt. 100%[===================>]   4.50M  12.0MB/s    in 0.4

In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from pprint import pprint
from tqdm.notebook import tqdm
import string


nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('stopwords', quiet=True)

True

In [3]:
with open('corpus_wsd_50k.txt', 'r', encoding='utf-8') as f:
    corpus = f.read().split('\n\n')

print(f'Загружено {len(corpus)} предложений')

Загружено 49453 предложений


In [4]:
processed_corpus = []
ambiguous_word_count = 0  # для статистики
punctuation_to_clean = {
    ' ,': ',',
    ' .': '.',
    ' ?': '?',
    ' !': '!',
    ' :': ':',
    ' ;': ';'}

for sent in corpus:
    if not sent:
        continue

    sentence_data = {'tokens': []}
    word_forms = []

    lines = sent.split('\n')
    for line in lines:
        parts = line.split('\t')
        if len(parts) != 3:
            continue

        sense_key, lemma, word_form = parts
        word_forms.append(word_form)

        token_info = {
            'word_form': word_form,
            'lemma': lemma,
            'sense': None
        }

        if sense_key:
            ambiguous_word_count += 1
            try:
                synset = wn.lemma_from_key(sense_key).synset()
                token_info['sense'] = {
                    'key': sense_key,
                    'definition': synset.definition()
                }
            except nltk.corpus.reader.wordnet.WordNetError:
                # Если ключ вдруг не нашёлся, значения не будет
                token_info['sense'] = {
                    'key': sense_key,
                    'definition': ''
                }

        sentence_data['tokens'].append(token_info)

    # Собираем текст всего предложения
    text = ' '.join(word_forms)
    # Удаляем лишние пробелы перед пунктуацией
    for punc_with_space, punc in punctuation_to_clean.items():
        text = text.replace(punc_with_space, punc)
    sentence_data['text'] = text

    processed_corpus.append(sentence_data)

In [5]:
# Проверяем, что получилось

print(f"Всего предложений: {len(processed_corpus)}")
print(f"Всего неоднозначных слов: {ambiguous_word_count}")

Всего предложений: 49452
Всего неоднозначных слов: 239913


In [6]:
# Примеры
pprint(processed_corpus[:5])

[{'text': 'How long has it been since you reviewed the objectives of your '
          'benefit and service program?',
  'tokens': [{'lemma': 'how', 'sense': None, 'word_form': 'How'},
             {'lemma': 'long',
              'sense': {'definition': 'primarily temporal sense; being or '
                                      'indicating a relatively great or '
                                      'greater than average duration or '
                                      'passage of time or a duration as '
                                      'specified',
                        'key': 'long%3:00:02::'},
              'word_form': 'long'},
             {'lemma': 'have', 'sense': None, 'word_form': 'has'},
             {'lemma': 'it', 'sense': None, 'word_form': 'it'},
             {'lemma': 'be',
              'sense': {'definition': 'have the quality of being; (copula, '
                                      'used with an adjective or a predicate '
                                  

## 2. Алгоритм Леска с мерой Жаккарта

Здесь поэкспериментируем с разным размером окна, а также с удалением стоп-слов. Кроме того, проверим, что работает лучше: леммы или словоформы.

In [6]:
english_stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

In [7]:
def jaccard_similarity(set1, set2):
    """Вычисляет меру Джаккарда между двумя наборами слов."""
    if not set1 and not set2:
        return 0.0

    intersection = set1.intersection(set2)
    union = set1.union(set2)

    return len(intersection) / len(union)

In [8]:
def disambiguate_sentence_jaccard(tokens,
                                  remove_stop_words=False,
                                  window_size=None,
                                  use_lemmas=False):
    """
    Предсказывает значение для многозначных слов в предложении с использованием алгоритма Леска
    и меры Джаккарда.
    """

    def _preprocess_definition(text, remove_stops=False, lemmatize=False):
        # Внутренняя функция для предобработки текста определения
        # Токенизация, очистка от пунктуации
        text_tokens = nltk.word_tokenize(text)
        clean_tokens = [
            token.lower() for token in text_tokens if token not in punctuation]

        # Лемматизация (если требуется)
        if lemmatize:
            processed_tokens = [lemmatizer.lemmatize(
                token) for token in clean_tokens]
        else:
            processed_tokens = clean_tokens

        # Удаление стоп-слов (если требуется)
        if remove_stops:
            processed_tokens = [
                token for token in processed_tokens if token not in english_stopwords]

        return set(processed_tokens)

    # Проходим по всем токенам в предложении
    for i, token in enumerate(tokens):
        # Обрабатываем только многозначные слова
        if not (token.get('sense') and token['sense'].get('key')):
            continue

        # Определяем контекст
        context_words = set()
        token_key = 'lemma' if use_lemmas else 'word_form'

        if window_size is None:
            # Если размер окна не задан, контекст - все предложение
            context_words_list = [
                t[token_key].lower() for t in tokens if t[token_key] not in punctuation]
            if remove_stop_words:
                context_words_list = [
                    w for w in context_words_list if w not in english_stopwords]
            context_words = set(context_words_list) - \
                {token[token_key].lower()}
        else:
            # Если окно задано, сначала отбираем чистые слова, потом берем окно
            context_words_list = []

            # Идем влево от целевого слова, собирая n чистых слов
            words_count_left = 0
            for j in range(i - 1, -1, -1):
                if words_count_left == window_size:
                    break

                word = tokens[j][token_key].lower()

                if word in punctuation:
                    continue
                if remove_stop_words and word in english_stopwords:
                    continue

                context_words_list.append(word)
                words_count_left += 1

            # Идем вправо от целевого слова, собирая n чистых слов
            words_count_right = 0
            for j in range(i + 1, len(tokens)):
                if words_count_right == window_size:
                    break

                word = tokens[j][token_key].lower()

                if word in punctuation:
                    continue
                if remove_stop_words and word in english_stopwords:
                    continue

                context_words_list.append(word)
                words_count_right += 1
            context_words = set(context_words_list)

        # Получаем все возможные значения для леммы
        possible_synsets = wn.synsets(token['lemma'])
        if not possible_synsets:
            continue

        best_sense = None
        max_score = -1

        # Находим ближайшее значение, сравнивая контекст с определениями
        for synset in possible_synsets:
            definition_words = _preprocess_definition(
                synset.definition(),
                remove_stops=remove_stop_words,
                lemmatize=use_lemmas
            )
            score = jaccard_similarity(context_words, definition_words)

            if score > max_score:
                max_score = score
                best_sense = synset

        # Сохраняем предсказанное значение
        if best_sense:
            token['predicted_sense'] = {
                'name': best_sense.name(),
                'definition': best_sense.definition()
            }

    return tokens

In [9]:
def process_corpus_jaccard(
        corpus,
        remove_stop_words=False,
        window_size=None,
        use_lemmas=False):
    """
    Применяет функцию дизамбигуации на основе Джаккарда для всех предложений в корпусе
    """
    processed_with_predictions = []
    for sentence in tqdm(
            corpus,
            desc=f"Обработка (window={window_size}, stopwords={not remove_stop_words}, lemmas={use_lemmas})"):
        sentence_copy = {
            'text': sentence['text'],
            'tokens': [t.copy() for t in sentence['tokens']]
        }

        disambiguated_tokens = disambiguate_sentence_jaccard(
            sentence_copy['tokens'],
            remove_stop_words,
            window_size,
            use_lemmas
        )
        sentence_copy['tokens'] = disambiguated_tokens
        processed_with_predictions.append(sentence_copy)

    return processed_with_predictions

In [7]:
def evaluate(corpus_with_predictions):
    """
    Вычисляет accuracy и сохраняет примеры с ошибками
    """
    total = 0
    correct = 0
    errors = []

    for sentence in corpus_with_predictions:
        for token in sentence['tokens']:
            if token.get('sense') and token['sense'].get(
                    'key') and token.get('predicted_sense'):
                total += 1

                gold_key = token['sense']['key']
                predicted_name = token['predicted_sense']['name']

                try:
                    gold_synset = wn.lemma_from_key(gold_key).synset()
                    predicted_synset = wn.synset(predicted_name)

                    if gold_synset == predicted_synset:
                        correct += 1
                    else:
                        errors.append({
                            'text': sentence['text'],
                            'word': token['word_form'],
                            'lemma': token['lemma'],
                            'gold_sense': {'key': gold_key, 'definition': token['sense']['definition']},
                            'predicted_sense': token['predicted_sense']
                        })
                except Exception as e:
                    pass

    accuracy = correct / total if total > 0 else 0.0
    return {'average_accuracy': accuracy, 'errors': errors}

### Эксперимент 1: всё предложение, не убираем стоп-слова и не лемматизируем

In [11]:
corpus_pred_1 = process_corpus_jaccard(
    processed_corpus,
    remove_stop_words=False,
    window_size=None,
    use_lemmas=False)
results_1 = evaluate(corpus_pred_1)
print(
    f"Accuracy (полный контекст, со стоп-словами, без лемматизации): {results_1['average_accuracy']:.4f}")

Обработка (window=None, stopwords=True, lemmas=False):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (полный контекст, со стоп-словами, без лемматизации): 0.3025


### Эксперимент 2: всё предложение, убираем стоп-слова, не лемматизируем

In [12]:
corpus_pred_2 = process_corpus_jaccard(
    processed_corpus,
    remove_stop_words=True,
    window_size=None,
    use_lemmas=False)
results_2 = evaluate(corpus_pred_2)
print(
    f"Accuracy (полный контекст, без стоп-слов, без лемматизации): {results_2['average_accuracy']:.4f}")

Обработка (window=None, stopwords=False, lemmas=False):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (полный контекст, без стоп-слов, без лемматизации): 0.4532


### Эксперимент 3: всё предложение, убираем стоп-слова, лемматизируем

In [13]:
corpus_pred_3 = process_corpus_jaccard(
    processed_corpus,
    remove_stop_words=True,
    window_size=None,
    use_lemmas=True)
results_3 = evaluate(corpus_pred_3)
print(
    f"Accuracy (полный контекст, без стоп-слов, с лемматизацией): {results_2['average_accuracy']:.4f}")

Обработка (window=None, stopwords=False, lemmas=True):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (полный контекст, без стоп-слов, с лемматизацией): 0.4532


### Эксперименты с разным размером контекстного окна

In [18]:
best_accuracy = -1
best_params = {}
best_results = {}

for window in range(3, 8):
    for remove_stops in [False, True]:
        for use_lemmas in [False, True]:
            corpus_pred = process_corpus_jaccard(
                processed_corpus,
                remove_stop_words=remove_stops,
                window_size=window,
                use_lemmas=use_lemmas)
            results = evaluate(corpus_pred)
            accuracy = results['average_accuracy']

            print(
                f"Accuracy (window={window}, remove_stopwords={remove_stops}, use_lemmas={use_lemmas}): {accuracy:.4f}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = {
                    'window_size': window,
                    'remove_stop_words': remove_stops,
                    'use_lemmas': use_lemmas}
                best_results = results

Обработка (window=3, stopwords=True, lemmas=False):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (window=3, remove_stopwords=False, use_lemmas=False): 0.3366


Обработка (window=3, stopwords=True, lemmas=True):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (window=3, remove_stopwords=False, use_lemmas=True): 0.3338


Обработка (window=3, stopwords=False, lemmas=False):   0%|          | 0/49452 [00:00<?, ?it/s]

...

Accuracy (window=7, remove_stopwords=False, use_lemmas=True): 0.3094


Обработка (window=7, stopwords=False, lemmas=False):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (window=7, remove_stopwords=True, use_lemmas=False): 0.4565


Обработка (window=7, stopwords=False, lemmas=True):   0%|          | 0/49452 [00:00<?, ?it/s]

Accuracy (window=7, remove_stopwords=True, use_lemmas=True): 0.4464


In [19]:
print("\n--- Лучший эксперимент ---")
print(f"Лучшая Accuracy: {best_accuracy:.4f}")
print(f"Лучшие параметры: {best_params}")

print("\n--- 10 ошибок для лучшей модели ---")
for i, error in enumerate(best_results['errors'][:10]):
    print(f"Ошибка #{i+1}")
    print(f"Контекст: ...{error['text']}...")
    print(f"Слово: '{error['word']}' (лемма: '{error['lemma']}')")
    print(
        f"  - Настоящее значение     : [{error['gold_sense']['key']}] {error['gold_sense']['definition']}")
    print(
        f"  - Предсказанное: [{error['predicted_sense']['name']}] {error['predicted_sense']['definition']}\n")


--- Лучший эксперимент ---
Лучшая Accuracy: 0.4659
Лучшие параметры: {'window_size': 3, 'remove_stop_words': True, 'use_lemmas': False}

--- 10 ошибок для лучшей модели ---
Ошибка #1
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'long' (лемма: 'long')
  - Настоящее значение     : [long%3:00:02::] primarily temporal sense; being or indicating a relatively great or greater than average duration or passage of time or a duration as specified
  - Предсказанное: [hanker.v.01] desire strongly or persistently

Ошибка #2
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'been' (лемма: 'be')
  - Настоящее значение     : [be%2:42:03::] have the quality of being; (copula, used with an adjective or a predicate noun)
  - Предсказанное: [beryllium.n.01] a light strong brittle grey toxic bivalent metallic element

Ошибка #3
Контекст: ...How long has it been since you r

Во-первых, удаление стоп-слов существенно улучшает результат (больше чем на 15%), что для lexical-based-подхода более чем ожидаемо. А вот использование лемм вместо словоформ, наоборот, результат ухудшает. Связано это, видимо, с плохим качеством лемматизатора на основе Wordnet (который использовался, чтобы лемматизировать токены для определений синсетов), а также с тем, что тексты на английском языке из-за аналитической природы языка не нуждаются в этом шаге препроцессинга в той же степени, как, например, русскоязычные.

Также мы увидели, что при использовании данного алгоритма самым эффективным оказалось брать минимальное контекстное окно (3), хотя, судя по метрикам, его размер вообще не особо значительно влиял на результат — решающее значение имели только стоп-слова.

В целом, результат довольно плохой, мы не достигли даже 50% точности. Видно, что, если пересечений контекста и определения нет вообще, выбирается самое короткое, а иначе мы полагаемся на просто какие-то случайные слова, которым повезло попасть в пересечение (чаще всего, 1-2). Однако учитывая то, что это даже не бинарная, а многоклассовая классификация, результат совсем уж бессмысленным назвать нельзя.

## 3. Алгоритм Леска с семантическими эмбеддингами

Возьмём три разные модели типа Sentence-Transformers и проверим, как они справляются с нашей задачей. Никакой предобработки не будет, так как эти модели учились на реальных текстах и не видели текста без пунктуации, стоп-слов или с одними леммами. Окно тоже будет только всё предложение, опять же, из-за природы текстов, на которых такие модели обучались.

In [8]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from collections import defaultdict

2025-10-03 12:24:13.616357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759483453.752394    5477 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759483453.789776    5477 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759483454.066813    5477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759483454.066874    5477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759483454.066879    5477 computation_placer.cc:177] computation placer alr

In [9]:
def get_embeddings(corpus, model_name, query_prefix=None, doc_prefix=None):
    """
    Считает эмбеддинги для всех предложений и всех определений многозначных слов в корпусе.
    """
    # Загружаем модель
    model = SentenceTransformer(model_name)

    # Получаем эмбеддинги для всех предложений
    sentence_texts = [sentence['text'] for sentence in corpus]
    if query_prefix:
        sentence_texts = [f"{query_prefix}{text}" for text in sentence_texts]

    sentence_embeddings = model.encode(
        sentence_texts,
        show_progress_bar=True,
        normalize_embeddings=True)

    # Добавляем эмбеддинги в исходный корпус
    for i, sentence in enumerate(corpus):
        sentence['embedding'] = sentence_embeddings[i]

    # Собираем все уникальные многозначные слова и их значения
    unique_polysemous_lemmas = set()
    for sentence in corpus:
        for token in sentence['tokens']:
            if token.get('sense') and token['sense'].get('key'):
                unique_polysemous_lemmas.add(token['lemma'])

    # Получаем эмбеддинги для каждого значения
    sense_data = defaultdict(list)
    definitions_to_encode = []
    sense_order_map = []

    for lemma in tqdm(unique_polysemous_lemmas, desc="Сбор толкований"):
        synsets = wn.synsets(lemma)
        for synset in synsets:
            definitions_to_encode.append(synset.definition())
            sense_order_map.append({'lemma': lemma, 'synset': synset})

    if doc_prefix:
        definitions_to_encode = [
            f"{doc_prefix}{text}" for text in definitions_to_encode]

    definition_embeddings = model.encode(
        definitions_to_encode,
        show_progress_bar=True,
        normalize_embeddings=True)

    # Сопоставляем эмбеддинги с их значениями
    for i, embedding in enumerate(definition_embeddings):
        info = sense_order_map[i]
        sense_data[info['lemma']].append({
            'synset': info['synset'],
            'embedding': embedding
        })

    return corpus, sense_data

In [10]:
def disambiguate_with_embeddings(corpus_with_embeddings, sense_embeddings):
    """
    Предсказывает значения для многозначных слов в корпусе, используя алгоритм Леска и косинусное сходство эмбеддингов.
    """
    for sentence in tqdm(
            corpus_with_embeddings,
            desc="Disambiguating sentences"):
        sentence_emb = sentence['embedding'].reshape(1, -1)

        for token in sentence['tokens']:
            if token.get('sense') and token['sense'].get('key'):
                lemma = token['lemma']

                # Проверяем, есть ли у нас эмбеддинги для значений этого слова
                if lemma not in sense_embeddings or not sense_embeddings[lemma]:
                    continue

                possible_senses = sense_embeddings[lemma]

                # Собираем эмбеддинги всех возможных значений в матрицу
                definition_embs = np.array(
                    [sense['embedding'] for sense in possible_senses])

                # Рассчитываем косинусную близость между эмбеддингом предложения
                # и эмбеддингами всех определений
                similarities = cosine_similarity(sentence_emb, definition_embs)

                # Находим значение с наибольшей близостью
                best_sense_index = np.argmax(similarities)
                best_sense = possible_senses[best_sense_index]['synset']

                # Сохраняем предсказанное значение
                token['predicted_sense'] = {
                    'name': best_sense.name(),
                    'definition': best_sense.definition()
                }

    return corpus_with_embeddings

### Эксперимент 1: all-mpnet-base-v2

Оригинальная моделька от авторов Sentence-Transformers, работает с некоторым качеством, но в 2025 году уже есть масса сильно лучших (и больших) моделей.

In [14]:
model_name_1 = "sentence-transformers/all-mpnet-base-v2"

# Получаем эмбеддинги данной моделью
corpus_with_embs_1, senses_with_embs_1 = get_embeddings(
    processed_corpus,
    model_name=model_name_1,
    query_prefix=None,
    doc_prefix=None
)

# 2. Предсказываем значения, используя эмбеддинги
corpus_pred_1 = disambiguate_with_embeddings(
    corpus_with_embs_1, senses_with_embs_1)

# Оцениваем
results_1 = evaluate(corpus_pred_1)
print(f"\nAccuracy для {model_name_1}: {results_1['average_accuracy']:.4f}\n")

print("--- 10 ошибок ---")
for i, error in enumerate(results_1['errors'][:10]):
    print(f"Ошибка #{i+1}")
    print(f"Контекст: ...{error['text']}...")
    print(f"Слово: '{error['word']}' (lemma: '{error['lemma']}')")
    print(
        f"  - Настоящее значение: [{error['gold_sense']['key']}] {error['gold_sense']['definition']}")
    print(
        f"  - Предсказанное значение: [{error['predicted_sense']['name']}] {error['predicted_sense']['definition']}\n")

Batches:   0%|          | 0/1546 [00:00<?, ?it/s]

Сбор толкований:   0%|          | 0/20387 [00:00<?, ?it/s]

Batches:   0%|          | 0/2073 [00:00<?, ?it/s]

Disambiguating sentences:   0%|          | 0/49452 [00:00<?, ?it/s]


Accuracy для sentence-transformers/all-mpnet-base-v2: 0.3708

--- 10 ошибок ---
Ошибка #1
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'long' (lemma: 'long')
  - Настоящее значение: [long%3:00:02::] primarily temporal sense; being or indicating a relatively great or greater than average duration or passage of time or a duration as specified
  - Предсказанное значение: [long.r.01] for an extended time or at a distant time

Ошибка #2
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'been' (lemma: 'be')
  - Настоящее значение: [be%2:42:03::] have the quality of being; (copula, used with an adjective or a predicate noun)
  - Предсказанное значение: [be.v.10] spend or use time

Ошибка #3
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'reviewed' (lemma: 'review')
  - Настоящее значение: [re

### Эксперимент 2: e5-large-v2

Хорошая, индустриальная модель, которая ещё в конце 2024 года была SOTA для векторного поиска (для английского языка), да и сейчас остаётся очень хорошей для индустриальных задач.

In [14]:
model_name_2 = "intfloat/e5-large-v2"

corpus_with_embs_2, senses_with_embs_2 = get_embeddings(
    processed_corpus,
    model_name=model_name_2,
    query_prefix="query: ",
    doc_prefix="passage: "
)

corpus_pred_2 = disambiguate_with_embeddings(
    corpus_with_embs_2, senses_with_embs_2)

results_2 = evaluate(corpus_pred_2)
print(f"\nAccuracy для {model_name_2}: {results_2['average_accuracy']:.4f}\n")

print("--- 10 ошибок ---")
for i, error in enumerate(results_2['errors'][:10]):
    print(f"Ошибка #{i+1}")
    print(f"Контекст: ...{error['text']}...")
    print(f"Слово: '{error['word']}' (lemma: '{error['lemma']}')")
    print(
        f"  - Настоящее значение: [{error['gold_sense']['key']}] {error['gold_sense']['definition']}")
    print(
        f"  - Предсказанное значение: [{error['predicted_sense']['name']}] {error['predicted_sense']['definition']}\n")

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1546 [00:00<?, ?it/s]

Сбор толкований:   0%|          | 0/20387 [00:00<?, ?it/s]

Batches:   0%|          | 0/2073 [00:00<?, ?it/s]

Disambiguating sentences:   0%|          | 0/49452 [00:00<?, ?it/s]


Accuracy для intfloat/e5-large-v2: 0.3647

--- 10 ошибок ---
Ошибка #1
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'long' (lemma: 'long')
  - Настоящее значение: [long%3:00:02::] primarily temporal sense; being or indicating a relatively great or greater than average duration or passage of time or a duration as specified
  - Предсказанное значение: [long.r.01] for an extended time or at a distant time

Ошибка #2
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'been' (lemma: 'be')
  - Настоящее значение: [be%2:42:03::] have the quality of being; (copula, used with an adjective or a predicate noun)
  - Предсказанное значение: [be.v.10] spend or use time

Ошибка #3
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'reviewed' (lemma: 'review')
  - Настоящее значение: [review%2:31:00::] loo

### Эксперимент 3: multilingual-e5-large-instruct

Кажется, лучшее, что есть в серии E5, не считая огромных LLM-based-моделек. Особенность модели в том, что она поддерживает короткие инструкции, которые, в теории, должны модифицировать её эмбеддинги под задачу. Здесь мы используем этот факт и составляем инструкцию для задачи WSD (в стиле тех, на которых модель училась).

In [14]:
model_name_3 = "intfloat/multilingual-e5-large-instruct"
instruction_3 = "Given a sentence, retrieve the definition of a word as it is used in that sentence."

corpus_with_embs_3, senses_with_embs_3 = get_embeddings(
    processed_corpus,
    model_name=model_name_3,
    query_prefix=f"Instruct: {instruction_3}\nQuery: ",
    doc_prefix=None
)

corpus_pred_3 = disambiguate_with_embeddings(
    corpus_with_embs_3, senses_with_embs_3)

results_3 = evaluate(corpus_pred_3)
print(f"\nAccuracy для {model_name_3}: {results_3['average_accuracy']:.4f}\n")

print("--- 10 ошибок ---")
for i, error in enumerate(results_3['errors'][:10]):
    print(f"Ошибка #{i+1}")
    print(f"Контекст: ...{error['text']}...")
    print(f"Слово: '{error['word']}' (lemma: '{error['lemma']}')")
    print(
        f"  - Настоящее значение: [{error['gold_sense']['key']}] {error['gold_sense']['definition']}")
    print(
        f"  - Предсказанное значение: [{error['predicted_sense']['name']}] {error['predicted_sense']['definition']}\n")

README.md: 0.00B [00:00, ?B/s]

Batches:   0%|          | 0/1546 [00:00<?, ?it/s]

Сбор толкований:   0%|          | 0/20387 [00:00<?, ?it/s]

Batches:   0%|          | 0/2073 [00:00<?, ?it/s]

Disambiguating sentences:   0%|          | 0/49452 [00:00<?, ?it/s]


Accuracy для intfloat/multilingual-e5-large-instruct: 0.4061

--- 10 ошибок ---
Ошибка #1
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'reviewed' (lemma: 'review')
  - Настоящее значение: [review%2:31:00::] look at again; examine again
  - Предсказанное значение: [review.v.05] look back upon (a period of time, sequence of events); remember

Ошибка #2
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'objectives' (lemma: 'objective')
  - Настоящее значение: [objective%1:09:00::] the goal intended to be attained (and which is believed to be attainable)
  - Предсказанное значение: [objective.a.02] serving as or indicating the object of a verb or of certain prepositions and used for certain other purposes

Ошибка #3
Контекст: ...How long has it been since you reviewed the objectives of your benefit and service program?...
Слово: 'benefit' (lemma: 'benefit'

Результаты очень интересные. Получилось, что все наши модели дают худший результат, чем мера Жаккарда. Правда, если посмотреть на примеры ошибок, видно, что здесь намного чаще выбираются не совсем «левые» толкования, а похожие (иногда даже спорно и непонятно, не права ли модель на самом деле).

Так, например, для слова *long* в контексте *How long has it been since you reviewed the objectives of your benefit and service program?* эталонное значение такое: 'primarily temporal sense; being or indicating a relatively great or greater than average duration or passage of time or a duration as specified'. Модель предсказала другое: 'for an extended time or at a distant time' — и я не уверен, что не могу с ней не согласиться. То же, например, можно сказать и для слова `review`.

Интересно, что хоть instruct-модель и обогнала все другие, обычная E5-Large-V2 показала худший результат, чем более маленькая и старая all-mpnet-base-v2. Возможно, дело в том, что множество индустриальных бенчмарков ухудшили непосредственно NLI модели, ну или в чём-то ещё.

Однако в целом видно, что все эти модели не обучены сопоставлять контексты с толкованиями слов, да ещё и когда целевое слово, в общем-то, неизвестно (может быть, надо было добавлять его в начало или конец текста или в инструкцию в последнем случае, но эта мысль слишком поздно пришла мне в голову).

Однако нам никто не мешает дообучить любую из этих моделек, сближая эмбеддинги контекстов и соответствующих толкований (из любого словаря), и тогда я больше чем уверен, что результат будет очень хорошим. То есть, выходит, что мера Жаккарда хороша для unsupervised-случаев, когда у нас нет соответствующих размеченных данных (или ресурсов для дообучения), но и качество этот метод даёт невысокое, то, что мы получили, в каком-то смысле потолок, непонятно, как улучшить результат. Семантические эмбеддеры, несмотря на более низкий результат для предобученных моделях, имеют гораздо больший потенциал, если у нас есть данные и ресурсы, чтобы их дообучить.

# Задание 2 (5 балла)
Попробуйте разные алгоритмы кластеризации на датасете - `https://github.com/nlpub/russe-wsi-kit/blob/initial/data/main/wiki-wiki/train.csv`

Используйте код из семинара как основу. Используйте ARI как метрику качества.

Попробуйте все 4 алгоритма кластеризации, про которые говорилось на семинаре. Для каждого из алгоритмов попробуйте настраивать гиперпараметры (посмотрите их в документации). Прогоните как минимум 5 экспериментов (не обязательно успешных) с разными параметрами на каждый алгоритме кластеризации и оцените: качество кластеризации, скорость работы, интуитивность параметров.

Помимо этого также выберите 1 дополнительный алгоритм кластеризации отсюда - https://scikit-learn.org/stable/modules/clustering.html , опишите своими словами принцип его работы  и проделайте аналогичные эксперименты. 

## 1. Загружаем данные и смотрим статистику

In [1]:
import pandas as pd


url = 'https://raw.githubusercontent.com/nlpub/russe-wsi-kit/initial/data/main/wiki-wiki/train.csv'
df = pd.read_csv(
    url,
    sep='\t',
    header=0,
    names=[
        'context_id',
        'word',
        'gold_sense_id',
        'predict_sense_id',
        'positions',
        'context'])

In [3]:
total_contexts = len(df)
unique_words = df['word'].unique()
num_unique_words = len(unique_words)

# Группируем по слову, чтобы найти количество уникальных значений для
# каждого слова
senses_per_word = df.groupby('word')['gold_sense_id'].nunique()

avg_meanings_per_word = senses_per_word.mean()
avg_contexts_per_word = total_contexts / num_unique_words

print("--- Статистика датасета ---")
print(f"Всего контекстов: {total_contexts}")
print(f"Количество многозначных слов: {num_unique_words}")
print(f"Среднее количество контекстов на слово: {avg_contexts_per_word:.2f}")
print(f"Среднее количество значений для слова: {avg_meanings_per_word:.2f}")

--- Статистика датасета ---
Всего контекстов: 439
Количество многозначных слов: 4
Среднее количество контекстов на слово: 109.75
Среднее количество значений для слова: 2.00


## 2. Экспериментируем с кластеризацией

In [15]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics import adjusted_rand_score
import numpy as np
from tqdm.notebook import tqdm
import time
import collections

In [5]:
def generate_embeddings(df, model_name, instruction=None):
    """
    Вычисляет эмбеддинги для всех контекстов в датафрейме с использованием указанной модели.
    """
    model = SentenceTransformer(model_name)

    contexts = df['context'].tolist()

    if instruction:
        contexts = [
            f"Instruct: {instruction}\nQuery: {ctx}" for ctx in contexts]

    embeddings = model.encode(
        contexts,
        show_progress_bar=True,
        normalize_embeddings=True)

    df_embedded = df.copy()
    df_embedded['embedding'] = list(embeddings)

    return df_embedded

In [6]:
wsi_instruction = "Identify the meaning of a word based on the sentence it appears in"

df_embedded = generate_embeddings(
    df,
    'intfloat/multilingual-e5-large-instruct',
    instruction=wsi_instruction)

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

### Эксперимент 1: K-means

Здесь мы можем задать количество кластеров (подсказано данными), а также количество запусков. По идеи, чем больше запусков, тем больше шансов на лучший исход, но и работать будет дольше.

In [24]:
from sklearn.cluster import KMeans

print("--- Running K-Means Experiments ---")

# Гиперпараметры
kmeans_params_to_test = [
    {'n_clusters': 2, 'n_init': 'auto', 'random_state': 42},
    {'n_clusters': 2, 'n_init': 'auto', 'algorithm': 'elkan', 'random_state': 42},
    {'n_clusters': 3, 'n_init': 'auto', 'random_state': 42},
    {'n_clusters': 4, 'n_init': 'auto', 'random_state': 42},
    {'n_clusters': 2, 'n_init': 1, 'init': 'random', 'random_state': 42},
    {'n_clusters': 2, 'n_init': 10, 'init': 'random', 'random_state': 42},
    {'n_clusters': 2, 'n_init': 20, 'random_state': 42},
    {'n_clusters': 2, 'n_init': 30, 'random_state': 42}
]

unique_words = df_embedded['word'].unique()
kmeans_results = []

for params in tqdm(kmeans_params_to_test, desc="K-Means Hyperparameters"):
    print(f"\n--- Testing params: {params} ---")
    total_ari = 0
    total_time = 0

    for word in unique_words:
        word_df = df_embedded[df_embedded['word'] == word]
        embeddings = np.vstack(word_df['embedding'].values)
        gold_labels = word_df['gold_sense_id'].values

        clusterer = KMeans(**params)

        start_time = time.time()
        predicted_labels = clusterer.fit_predict(embeddings)
        end_time = time.time()

        # Статистика по каждому слову
        gold_counts = collections.Counter(gold_labels)
        pred_counts = collections.Counter(predicted_labels)
        print(f"  Word: '{word}'")
        print(
            f"    - Clusters (Gold/Pred): {len(gold_counts)} / {len(pred_counts)}")
        print(
            f"    - Avg. Contexts/Cluster (Gold/Pred): {np.mean(list(gold_counts.values())):.1f} / {np.mean(list(pred_counts.values())):.1f}")

        total_time += (end_time - start_time)
        total_ari += adjusted_rand_score(gold_labels, predicted_labels)

    avg_ari = total_ari / len(unique_words)
    kmeans_results.append(
        {'params': params, 'ari': avg_ari, 'time': total_time})
    print(
        f"  --------------------\n  Avg ARI: {avg_ari:.4f} | Total Time: {total_time:.4f}s")

# Финальные результаты
best_ari_run = max(kmeans_results, key=lambda x: x['ari'])
fastest_run = min(kmeans_results, key=lambda x: x['time'])

print("\n--- K-Means Final Summary ---")
print(
    f"Best ARI Score: {best_ari_run['ari']:.4f} with params: {best_ari_run['params']}")
print(
    f"Fastest Run: {fastest_run['time']:.4f}s with params: {fastest_run['params']}")

--- Running K-Means Experiments ---


K-Means Hyperparameters:   0%|          | 0/8 [00:00<?, ?it/s]


--- Testing params: {'n_clusters': 2, 'n_init': 'auto', 'random_state': 42} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 69.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 55.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 67.5
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 28.0
  --------------------
  Avg ARI: 0.7200 | Total Time: 0.0142s

--- Testing params: {'n_clusters': 2, 'n_init': 'auto', 'algorithm': 'elkan', 'random_state': 42} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 69.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 55.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 67.5
  Word: 'бор'
    - Clusters (Gold/Pred)

### Эксперимент 2: DBSCAN

Здесь ключевой гиперпараметр — `eps`, который по сути влияет на то, какой разброс дистанции будет учитываться при кластеризации. Учитывая, что наша моделька обучена так, что значения косинусной близости в основном разбросаны в диапазоне 0.7-1.0 (из документации), имеет смысл брать очень маленькие значения. Ну и ещё можно настроить минимальное количество сэмплов в каждом кластере.

In [25]:
from sklearn.cluster import DBSCAN

print("\n--- Running DBSCAN Experiments ---")

# Гиперпараметры
dbscan_params_to_test = [
    {'eps': 0.05, 'min_samples': 3, 'metric': 'cosine'},
    {'eps': 0.1, 'min_samples': 3, 'metric': 'cosine'},
    {'eps': 0.12, 'min_samples': 3, 'metric': 'cosine'},
    {'eps': 0.05, 'min_samples': 5, 'metric': 'cosine'},
    {'eps': 0.1, 'min_samples': 5, 'metric': 'cosine'},
]

dbscan_results = []

for params in tqdm(dbscan_params_to_test, desc="DBSCAN Hyperparameters"):
    print(f"\n--- Testing params: {params} ---")
    total_ari = 0
    total_time = 0

    for word in unique_words:
        word_df = df_embedded[df_embedded['word'] == word]
        embeddings = np.vstack(word_df['embedding'].values)
        gold_labels = word_df['gold_sense_id'].values

        clusterer = DBSCAN(**params)

        start_time = time.time()
        predicted_labels = clusterer.fit_predict(embeddings)
        end_time = time.time()

        gold_counts = collections.Counter(gold_labels)
        pred_counts = collections.Counter(predicted_labels)
        num_pred_clusters = len(pred_counts) - (1 if -1 in pred_counts else 0)

        print(f"  Word: '{word}'")
        print(
            f"    - Clusters (Gold/Pred): {len(gold_counts)} / {num_pred_clusters} (+ {pred_counts.get(-1, 0)} noise points)")
        print(
            f"    - Avg. Contexts/Cluster (Gold/Pred): {np.mean(list(gold_counts.values())):.1f} / {np.mean([c for k, c in pred_counts.items() if k != -1]) if num_pred_clusters > 0 else 0:.1f}")

        total_time += (end_time - start_time)
        if len(set(predicted_labels)) > 1:
            total_ari += adjusted_rand_score(gold_labels, predicted_labels)
        else:
            total_ari += 0

    avg_ari = total_ari / len(unique_words)
    dbscan_results.append(
        {'params': params, 'ari': avg_ari, 'time': total_time})
    print(
        f"  --------------------\n  Avg ARI: {avg_ari:.4f} | Total Time: {total_time:.4f}s")

# Финальные результаты
best_ari_run_db = max(dbscan_results, key=lambda x: x['ari'])
fastest_run_db = min(dbscan_results, key=lambda x: x['time'])

print("\n--- DBSCAN Final Summary ---")
print(
    f"Best ARI Score: {best_ari_run_db['ari']:.4f} with params: {best_ari_run_db['params']}")
print(
    f"Fastest Run: {fastest_run_db['time']:.4f}s with params: {fastest_run_db['params']}")


--- Running DBSCAN Experiments ---


DBSCAN Hyperparameters:   0%|          | 0/5 [00:00<?, ?it/s]


--- Testing params: {'eps': 0.05, 'min_samples': 3, 'metric': 'cosine'} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 1 (+ 2 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 136.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2 (+ 6 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 52.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 11 (+ 11 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 11.3
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 4 (+ 6 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 12.5
  --------------------
  Avg ARI: 0.4018 | Total Time: 0.0138s

--- Testing params: {'eps': 0.1, 'min_samples': 3, 'metric': 'cosine'} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 1 (+ 0 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 138.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 1 (+ 0 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 110.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 1 (+ 0 

### Эксперимент 3: Affinity Propagation

Здесь два ключевых параметра: `damping` и `preference`. Последний непосредственно влияет на количество кластеров: если ставить отрицательные значения, кластеров будет поменьше, что нам как раз очень надо.

In [26]:
from sklearn.cluster import AffinityPropagation

print("\n--- Running Affinity Propagation Experiments ---")

# Гиперпараметры
ap_params_to_test = [
    {'damping': 0.5, 'random_state': 42},
    {'damping': 0.7, 'random_state': 42},
    {'damping': 0.9, 'random_state': 42},
    {'damping': 0.9, 'preference': -5, 'random_state': 42},
    {'damping': 0.9, 'preference': -10, 'random_state': 42},
    {'damping': 0.95, 'preference': -20, 'random_state': 42},
    {'damping': 0.95, 'preference': -25, 'random_state': 42},
    {'damping': 0.95, 'preference': -30, 'random_state': 42},
    {'damping': 0.9, 'preference': -50, 'random_state': 42},
    {'damping': 0.9, 'preference': -100, 'random_state': 42},
]

ap_results = []

for params in tqdm(ap_params_to_test, desc="AffinityProp Hyperparameters"):
    print(f"\n--- Testing params: {params} ---")
    total_ari = 0
    total_time = 0

    for word in unique_words:
        word_df = df_embedded[df_embedded['word'] == word]
        embeddings = np.vstack(word_df['embedding'].values)
        gold_labels = word_df['gold_sense_id'].values

        clusterer = AffinityPropagation(**params)

        start_time = time.time()
        predicted_labels = clusterer.fit_predict(embeddings)
        end_time = time.time()

        gold_counts = collections.Counter(gold_labels)
        pred_counts = collections.Counter(predicted_labels)
        print(f"  Word: '{word}'")
        print(
            f"    - Clusters (Gold/Pred): {len(gold_counts)} / {len(pred_counts)}")
        print(
            f"    - Avg. Contexts/Cluster (Gold/Pred): {np.mean(list(gold_counts.values())):.1f} / {np.mean(list(pred_counts.values())):.1f}")

        total_time += (end_time - start_time)
        if len(set(predicted_labels)) > 1:
            total_ari += adjusted_rand_score(gold_labels, predicted_labels)
        else:
            total_ari += 0

    avg_ari = total_ari / len(unique_words)
    ap_results.append({'params': params, 'ari': avg_ari, 'time': total_time})
    print(
        f"  --------------------\n  Avg ARI: {avg_ari:.4f} | Total Time: {total_time:.4f}s")

# Финальные результаты
best_ari_run_ap = max(ap_results, key=lambda x: x['ari'])
fastest_run_ap = min(ap_results, key=lambda x: x['time'])

print("\n--- Affinity Propagation Final Summary ---")
print(
    f"Best ARI Score: {best_ari_run_ap['ari']:.4f} with params: {best_ari_run_ap['params']}")
print(
    f"Fastest Run: {fastest_run_ap['time']:.4f}s with params: {fastest_run_ap['params']}")


--- Running Affinity Propagation Experiments ---


AffinityProp Hyperparameters:   0%|          | 0/10 [00:00<?, ?it/s]


--- Testing params: {'damping': 0.5, 'random_state': 42} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 27
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 5.1
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 22
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 5.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 25
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 5.4
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 15
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 3.7
  --------------------
  Avg ARI: 0.0685 | Total Time: 0.0808s

--- Testing params: {'damping': 0.7, 'random_state': 42} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 26
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 5.3
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 23
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 4.8
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 25
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 5.4
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 15
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 3.7

### Эксперимент 4: Agglomerative Clustering

Здесь, как и в случае с K-means, мы можем задать количество кластеров, а также тип связи. Кроме того, вместо количества кластеров мы можем задать разброс расстояния, и тогда алгоритм попытается определить количество кластеров самостоятельно.

In [27]:
from sklearn.cluster import AgglomerativeClustering

print("\n--- Running Agglomerative Clustering Experiments ---")

# Гиперпараметры
agg_params_to_test = [
    {'n_clusters': 2, 'linkage': 'ward'},
    {'n_clusters': 2, 'linkage': 'complete'},
    {'n_clusters': 2, 'linkage': 'average'},
    {'n_clusters': 3, 'linkage': 'ward'},
    {'n_clusters': 4, 'linkage': 'ward'},
    {'n_clusters': None, 'distance_threshold': 0.1, 'linkage': 'complete', 'metric': 'cosine'},
    {'n_clusters': None, 'distance_threshold': 0.2, 'linkage': 'complete', 'metric': 'cosine'},
    {'n_clusters': None, 'distance_threshold': 0.3, 'linkage': 'complete', 'metric': 'cosine'},
    {'n_clusters': None, 'distance_threshold': 0.5, 'linkage': 'average', 'metric': 'cosine'},
]

agg_results = []

for params in tqdm(agg_params_to_test, desc="Agglomerative Hyperparameters"):
    print(f"\n--- Testing params: {params} ---")
    total_ari = 0
    total_time = 0

    for word in unique_words:
        word_df = df_embedded[df_embedded['word'] == word]
        embeddings = np.vstack(word_df['embedding'].values)
        gold_labels = word_df['gold_sense_id'].values

        current_params = params.copy()
        if current_params.get('linkage') != 'ward':
            current_params['metric'] = 'cosine'

        clusterer = AgglomerativeClustering(**current_params)

        start_time = time.time()
        predicted_labels = clusterer.fit_predict(embeddings)
        end_time = time.time()

        gold_counts = collections.Counter(gold_labels)
        pred_counts = collections.Counter(predicted_labels)
        print(f"  Word: '{word}'")
        print(
            f"    - Clusters (Gold/Pred): {len(gold_counts)} / {len(pred_counts)}")
        print(
            f"    - Avg. Contexts/Cluster (Gold/Pred): {np.mean(list(gold_counts.values())):.1f} / {np.mean(list(pred_counts.values())):.1f}")

        total_time += (end_time - start_time)
        total_ari += adjusted_rand_score(gold_labels, predicted_labels)

    avg_ari = total_ari / len(unique_words)
    agg_results.append({'params': params, 'ari': avg_ari, 'time': total_time})
    print(
        f"  --------------------\n  Avg ARI: {avg_ari:.4f} | Total Time: {total_time:.4f}s")

# Финальные результаты
best_ari_run_agg = max(agg_results, key=lambda x: x['ari'])
fastest_run_agg = min(agg_results, key=lambda x: x['time'])

print("\n--- Agglomerative Clustering Final Summary ---")
print(
    f"Best ARI Score: {best_ari_run_agg['ari']:.4f} with params: {best_ari_run_agg['params']}")
print(
    f"Fastest Run: {fastest_run_agg['time']:.4f}s with params: {fastest_run_agg['params']}")


--- Running Agglomerative Clustering Experiments ---


Agglomerative Hyperparameters:   0%|          | 0/9 [00:00<?, ?it/s]


--- Testing params: {'n_clusters': 2, 'linkage': 'ward'} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 69.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 55.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 67.5
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 28.0
  --------------------
  Avg ARI: 0.8958 | Total Time: 0.0154s

--- Testing params: {'n_clusters': 2, 'linkage': 'complete'} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 69.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 55.0
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 67.5
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 2
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 

### Эксперимент 5: HDBSCAN

HDBSCAN (Иерархический DBSCAN) — это усовершенствованная версия алгоритма DBSCAN. Он также ищет кластеры как области с высокой плотностью точек, но делает это более гибко. Вместо того чтобы искать кластеры на одном уровне плотности, который нужно задать вручную (параметр eps в DBSCAN), HDBSCAN рассматривает все возможные плотности одновременно. Он строит иерархию вложенных друг в друга кластеров, а затем выбирает из неё самые стабильные и устойчивые группы.
Основное отличие от DBSCAN заключается в том, что HDBSCAN не требует подбора параметра eps, что делает его способным находить кластеры разной плотности в одном и том же наборе данных. Главное преимущество этого алгоритма — он более надёжен и прост в настройке, так как требует подбора всего одного интуитивно понятного параметра (min_cluster_size). Но мы будем подбирать ещё и `min_samples`, а также попробуем две разные метрики.

Мы не будем использовать реализацию из Sklearn, а воспользуемся той, что есть в отдельной библиотеке `hdbscan`: она более правильная, простая в настройке и в целом «продовая».

In [28]:
from hdbscan import HDBSCAN

print("\n--- Running HDBSCAN Experiments ---")

# Гиперпараметры
hdbscan_params_to_test = [
    {'min_cluster_size': 5, 'min_samples': 5, 'metric': 'euclidean'},
    {'min_cluster_size': 8, 'min_samples': 8, 'metric': 'euclidean'},
    {'min_cluster_size': 10, 'min_samples': 5, 'metric': 'euclidean'},
    {'min_cluster_size': 12, 'metric': 'euclidean'},
    {'min_cluster_size': 12, 'min_samples': 5, 'metric': 'euclidean'},
    {'min_cluster_size': 10, 'metric': 'manhattan'},
    {'min_cluster_size': 15, 'metric': 'manhattan'},
    {'min_cluster_size': 5, 'min_samples': 3, 'metric': 'manhattan'},
    {'min_cluster_size': 10, 'min_samples': 5, 'metric': 'manhattan'},
    {'min_cluster_size': 12, 'min_samples': 5, 'metric': 'manhattan'},
]

hdbscan_results = []

for params in tqdm(hdbscan_params_to_test, desc="HDBSCAN Hyperparameters"):
    print(f"\n--- Testing params: {params} ---")
    total_ari = 0
    total_time = 0

    for word in unique_words:
        word_df = df_embedded[df_embedded['word'] == word]
        embeddings = np.vstack(word_df['embedding'].values)
        gold_labels = word_df['gold_sense_id'].values

        clusterer = HDBSCAN(**params)

        start_time = time.time()
        predicted_labels = clusterer.fit_predict(embeddings)
        end_time = time.time()

        gold_counts = collections.Counter(gold_labels)
        pred_counts = collections.Counter(predicted_labels)
        num_pred_clusters = len(pred_counts) - (1 if -1 in pred_counts else 0)

        print(f"  Word: '{word}'")
        print(
            f"    - Clusters (Gold/Pred): {len(gold_counts)} / {num_pred_clusters} (+ {pred_counts.get(-1, 0)} noise points)")
        print(
            f"    - Avg. Contexts/Cluster (Gold/Pred): {np.mean(list(gold_counts.values())):.1f} / {np.mean([c for k, c in pred_counts.items() if k != -1]) if num_pred_clusters > 0 else 0:.1f}")

        total_time += (end_time - start_time)
        if len(set(predicted_labels)) > 1:
            total_ari += adjusted_rand_score(gold_labels, predicted_labels)
        else:
            total_ari += 0

    avg_ari = total_ari / len(unique_words)
    hdbscan_results.append(
        {'params': params, 'ari': avg_ari, 'time': total_time})
    print(
        f"  --------------------\n  Avg ARI: {avg_ari:.4f} | Total Time: {total_time:.4f}s")

# Финальные результаты
best_ari_run_hdb = max(hdbscan_results, key=lambda x: x['ari'])
fastest_run_hdb = min(hdbscan_results, key=lambda x: x['time'])

print("\n--- HDBSCAN Final Summary ---")
print(
    f"Best ARI Score: {best_ari_run_hdb['ari']:.4f} with params: {best_ari_run_hdb['params']}")
print(
    f"Fastest Run: {fastest_run_hdb['time']:.4f}s with params: {fastest_run_hdb['params']}")


--- Running HDBSCAN Experiments ---


HDBSCAN Hyperparameters:   0%|          | 0/10 [00:00<?, ?it/s]


--- Testing params: {'min_cluster_size': 5, 'min_samples': 5, 'metric': 'euclidean'} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 4 (+ 33 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 26.2
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2 (+ 9 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 50.5
  Word: 'суда'
    - Clusters (Gold/Pred): 2 / 5 (+ 28 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 67.5 / 21.4
  Word: 'бор'
    - Clusters (Gold/Pred): 2 / 2 (+ 10 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 28.0 / 23.0
  --------------------
  Avg ARI: 0.5602 | Total Time: 0.1262s

--- Testing params: {'min_cluster_size': 8, 'min_samples': 8, 'metric': 'euclidean'} ---
  Word: 'замок'
    - Clusters (Gold/Pred): 2 / 2 (+ 52 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 69.0 / 43.0
  Word: 'лук'
    - Clusters (Gold/Pred): 2 / 2 (+ 22 noise points)
    - Avg. Contexts/Cluster (Gold/Pred): 55.0 / 44.0
  Word: 'суда'
    - Cluste

Выходит, что на нашей задаче лучше всего работают алгоритмы, в которых можно эксплицитно задать количество кластеров, что, наверное, ожидаемо. У наших данных нет разнообразия, у всех представленных слов по два значения, и для всех есть примерно равное количество контекстов. При этом Agglomerative Clustering работает лучше, чем K_means (~89.5% против ~83% при лучших подобранных гиперпараметрах), но K-means работает незначительно быстрее.

Проблема с этими алгоритмами в том, что для их реального применения для WSI мы должны точно знать, сколько предполагается значений у многозначного слова (скажем, инферировать эти значения из словарей или ещё как-то), в некотором смысле на наших данных задание количества кластеров сработало как "трейн на тесте", потому что мы знали, сколько значений есть у каждого слова, т.е. сколько должно быть кластеров, и по сути решали только задачу распределения контекстов по уже известным кластерам, что по сути уже практически классификация. Как подобрать количество кластеров для неизвестной выборки — не вполне понятно. Хотя, конечно, при намеренном задании кластеров 3-4 для этих алгоритмов значение метрики падало не так катострофически, так что можно попробовать. В целом же, видимо, надо опираться на какую-то статистику: например, взять много многозначных слов с известными контекстами и посмотреть, сколько в среднем у слова значений.

Что касается других алгоритмов, где мы не можем эксплицитно задавать количество кластеров, относительно неплохо справился только HDBSCAN. Там почти при всех параметрах получалось небольшое число кластеров, по которым распределялись контексты. Но проблема этого алгоритма в том, что значительное количество данных уходит в «шумовой» кластер, поэтому даже при верном определении количества кластеров туда просто не попадают многие контексты. Остальные же алгоритмы с задачей не справились практически совсем: DBSCAN показывает нулевые результаты, а AP — очень низкие, да и то — при очень тщательной настройке параметра `preference`, который позволяет уменьшить количество кластеров. Алгоритм Agglomerative Clustering можно настроить так, чтобы он сам пытался определять количество кластеров, основываясь на расстоянии, но это не сработало.

Про время работы: так как у нас очень мало данных, все алгоритмы работают быстро, и на пользовательском уровне разница не чувствуется. Однако при просмотре статистики видно, что HDBSCAN работает в ~10 раз медленнее, чем все остальные алгоритмы, тогда как остальные алгоритмы кластеризации находятся примерно в одном диапазоне. Самые быстрые — AP и KMeans.

Что касается интуитивности параметров, то, как кажется, количество кластеров у алгоритмов, где этот параметр есть, как раз и есть самый понятный и настраиваемый параметр (но у него есть проблемы, см. выше). Назначение других параметров плохо понятно без документации, для использования скорее удобнее запомнить, как они влияют на количество и плотность кластеров (как, например, `preference` в AP). Хотя параметры `min_cluster_size` и `min_samples` в HDBSCAN тоже выглядят довольно интуитивно, сразу +- понятно, за что они отвечают.

В целом можно сказать, что выбранная нами модель производит достаточно неплохие для различения контекстов эмбеддинги, но без точного количества этих значений кластеризация затруднена. Возможно также, что нам следовало использовать не эмбеддинг всего предложения, а только эмбеддинг, составленный из токенов таргетного слова — может быть, тогда бы результат был бы лучше. Кроме того, мы не экспериментировали с Umap и понижением размерности, в то время как для некоторых алгоритмов типа HDBSCAN это могло бы быть полезно. Ну и, наконец, никто не мешает дообучить Sentence-Transformers-модельку на задачу WSI. Та же задача sentence similarity, но тут мы просто сближаем эмбеддинги похожих контекстов. Опционально можно и отдалять эмбеддинги непохожих (контрастивное дообучение).